<a href="https://colab.research.google.com/github/Awwanna/PythonProDataScience/blob/master/CC_in_GEE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

import and installing libraries

In [4]:
import ee                           # Earth Engine
import geemap                       # geemap library
import matplotlib.pyplot as plt     # for figure visualisation
import pandas as pd                 # for data storage and manipulation
import numpy as np                  # array handling
#import ruptures as rpt              # for breakpoint detection


authentication and initialisation

In [5]:
ee.Authenticate()

In [6]:
ee.Initialize(project='ee-wildovaan')
print('Welcome to the Earth Engine Python API!')

Welcome to the Earth Engine Python API!


# Study area - Podgorica, Montenegro
- start and end date of wildfire: 20 Mar 2022 - 28 Mar 2022
- burnt area (ha): 1074
- one of the 130 wildfires in Montenegro in 2022
- main burnt ecosystems:
    - forest broadleaves 61.2 %,
    - transitional woodland-shurbland 36.2 %
    - agricultural areas 2.5 %

Define the study area

In [15]:
study_area = ee.Geometry
Map = geemap.Map()
display(Map)

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [16]:
# Load the drawn study area from the Map
study_area = ee.FeatureCollection(Map.draw_features)

# Or define it using Longitude and latitude coordinates
# roi = ee.Geometry.Point([longitude, latitude])

# inspect it
study_area

In [13]:
# Filter the Sentinel-2 image collection
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')# Import data
      .filterBounds(study_area)                                    # Spatial filter
      .filterDate('2022-03-01', '2022-05-01')               # Temporal filter
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))  # Metadata filter
      .sort('CLOUDY_PIXEL_PERCENTAGE'))                     # Sort data

# Print data to the Console
display(dataset)
display(dataset.first()) # Display image with lowest cloudy pixel percentage

In [20]:
# Add True color RGB composite to the map
Map.addLayer(
    dataset.first(),
    {'min': 0, 'max': 3000, 'bands': 'B4,B3,B2'},
    "S2 RGB Image"
)

# Add a false color composite
Map.addLayer(
    dataset.first(),
    {'min': 0, 'max': 3000, 'bands': 'B8,B4,B3'},
    "S2 NIR-R-G Image"
)

Map.centerObject(study_area)
display(Map)
Map.remove_last_drawn()

Map(bottom=193405.0, center=[42.7709455, 19.413185], controls=(WidgetControl(options=['position', 'transparent…

#Comparing of sentinel RGB images of the study area before and after the wildfire

In [40]:
# Define the burned area
burned_area = ee.Geometry.Polygon(
        [[[19.344864, 42.812025],
          [19.481506, 42.812025],
          [19.481506, 42.729866],
          [19.344864, 42.729866]]],
        )
style = {'color': 'eeb61bff', 'width': 3, 'lineType': 'solid', 'fillColor': '00000019'}
# Filter sentinel data - after wildfire and clouds
dataset_after_wildfire = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(burned_area)
      .filterDate('2022-03-29', '2022-04-30')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
      .sort('CLOUDY_PIXEL_PERCENTAGE'))
# Filter sentinel data - before wildfire and clouds
dataset_before_wildfire = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
      .filterBounds(burned_area)
      .filterDate('2022-03-29', '2022-04-30')
      .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
      .sort('CLOUDY_PIXEL_PERCENTAGE'))
# Add the first picture to map
Map1 = geemap.Map() # map for the before/after wildfire
Map1.addLayer(
    dataset_after_wildfire.first(),
    {'min': 0, 'max': 3000, 'bands': 'B4,B3,B2'},
    "S2 RGB Image"
)
Map1.addLayer(burned_area, vis_params=style)
Map1.centerObject(burned_area,10)
display(Map1)

Map(center=[42.77095672333046, 19.413185000001324], controls=(WidgetControl(options=['position', 'transparent_…

# Cloud masking using CloudScore+